In [1]:
# !pip install folktables

In [1]:
from folktables import ACSDataSource, ACSEmployment,ACSIncome
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import pickle

In [2]:
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
# acs_data = data_source.get_data(states=["AL", "AK", "AZ", "AR", "CA"], download=True)
# acs_data = data_source.get_data(download=True)

In [3]:
# state_codes = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
#                "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
#                "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
#                "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
#                "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

# state_codes = ["AL","FL","GA","CA","IL","OH","PA","TX", "NY", ]
state_codes = [ "CA","FL","GA","IL","OH","PA","TX","NY"]

dfs = {}
for state_code in state_codes:
    acs_data = data_source.get_data(states=[state_code],download=True)
    features, label, group = ACSIncome.df_to_pandas(acs_data)
    dfs[state_code] = (features, label)


In [4]:
all_len=[]
for state_code, (features, label) in dfs.items():
    all_len.append(len(label))
    print(f"State: {state_code}, Features Length: {len(features)}, Label Length: {len(label)}")

State: CA, Features Length: 195665, Label Length: 195665
State: FL, Features Length: 98925, Label Length: 98925
State: GA, Features Length: 50915, Label Length: 50915
State: IL, Features Length: 67016, Label Length: 67016
State: OH, Features Length: 62135, Label Length: 62135
State: PA, Features Length: 68308, Label Length: 68308
State: TX, Features Length: 135924, Label Length: 135924
State: NY, Features Length: 103021, Label Length: 103021


In [5]:
merge_dfs={}
sample_size=16000

for state_code, (features, label) in dfs.items():   

    merge_df = pd.concat([features, label], axis=1)
    merge_df = merge_df.dropna()
    print(len(merge_df))
    merge_df['PINCP'] = merge_df['PINCP'].replace({True: '>50K', False: '<=50K'})

    # Random sampling with maintaning the statistics 
    # merge_df['strat'] = merge_df['SEX'].astype(str) + '_' + merge_df['RELP'].astype(str)
    
    # merge_df['strat'] = merge_df['SEX'].astype(str) + '_' + merge_df['RELP'].astype(str) + '_' + merge_df['PINCP'].astype(str)
    merge_df['strat'] = (
        merge_df['COW'].astype(str) + '_' +
        merge_df['MAR'].astype(str) + '_' +
        merge_df['SEX'].astype(str) + '_' +
        merge_df['RAC1P'].astype(str) + '_' +
        merge_df['PINCP'].astype(str)
        )

    # Check for rare combinations (occur less than 2 times)
    #     
    
    combination_counts = merge_df['strat'].value_counts()
    rare_combinations = combination_counts[combination_counts < 2].index
    # print(len(merge_df))
    print("rare_combinations:",len(rare_combinations))

    # Remove rare combination:- E.g 2.0_14.0_False from FL
    
    merge_df = merge_df[~merge_df['strat'].isin(rare_combinations)]
    print(len(merge_df))

    sampled_df, _ = train_test_split(
    merge_df,
    train_size=sample_size,
    stratify=merge_df['strat'],
    random_state=42  
)

    sampled_df = sampled_df.drop('strat', axis=1)
    sampled_df = sampled_df.reset_index(drop=True)

    merge_dfs[state_code] = sampled_df
    # merge_dfs[state_code] = merge_df
    
for state_code, df in merge_dfs.items():
    label_counts = df['PINCP'].value_counts()
    print(f"State: {state_code}, df Length: {len(df)}, Label Counts: {label_counts.to_dict()}")

    # print(f"State: {state_code}, df Length: {len(df)}")

195665


rare_combinations: 138
195527
98925
rare_combinations: 141
98784
50915
rare_combinations: 135
50780
67016
rare_combinations: 121
66895
62135
rare_combinations: 124
62011
68308
rare_combinations: 115
68193
135924
rare_combinations: 143
135781
103021
rare_combinations: 130
102891
State: CA, df Length: 16000, Label Counts: {'<=50K': 9431, '>50K': 6569}
State: FL, df Length: 16000, Label Counts: {'<=50K': 10701, '>50K': 5299}
State: GA, df Length: 16000, Label Counts: {'<=50K': 10335, '>50K': 5665}
State: IL, df Length: 16000, Label Counts: {'<=50K': 9702, '>50K': 6298}
State: OH, df Length: 16000, Label Counts: {'<=50K': 10567, '>50K': 5433}
State: PA, df Length: 16000, Label Counts: {'<=50K': 10294, '>50K': 5706}
State: TX, df Length: 16000, Label Counts: {'<=50K': 10165, '>50K': 5835}
State: NY, df Length: 16000, Label Counts: {'<=50K': 9368, '>50K': 6632}


In [6]:
# Save the dictionary to a file
with open('dfs.pickle', 'wb') as f:
    pickle.dump(merge_dfs, f)

In [7]:
with open('dfs.pickle', 'rb') as f:
    dfs_loaded = pickle.load(f)

In [8]:
for state_code, df in dfs_loaded.items():
    print(df.head())
    print(f"State: {state_code}, df Length: {len(df)}")

   AGEP  COW  SCHL  MAR    OCCP   POBP  RELP  WKHP  SEX  RAC1P  PINCP
0  41.0  1.0  22.0  5.0   800.0    6.0   0.0  50.0  2.0    6.0  <=50K
1  58.0  1.0  18.0  5.0  4140.0  233.0   0.0   8.0  1.0    6.0  <=50K
2  45.0  1.0  20.0  5.0  9645.0    6.0   0.0  40.0  2.0    1.0   >50K
3  35.0  1.0  18.0  5.0  5240.0    6.0  17.0  37.0  1.0    1.0  <=50K
4  33.0  1.0  21.0  1.0  3255.0   48.0   1.0  26.0  2.0    1.0   >50K
State: CA, df Length: 16000
   AGEP  COW  SCHL  MAR    OCCP  POBP  RELP  WKHP  SEX  RAC1P  PINCP
0  59.0  2.0  19.0  1.0  5740.0  12.0   0.0  12.0  2.0    1.0  <=50K
1  21.0  1.0  18.0  5.0  4510.0  12.0   0.0  40.0  2.0    1.0  <=50K
2  38.0  3.0  20.0  1.0  5740.0  12.0   1.0  40.0  2.0    1.0  <=50K
3  49.0  2.0  20.0  5.0   420.0   6.0   0.0  50.0  2.0    1.0   >50K
4  37.0  2.0  20.0  1.0  3423.0  26.0   0.0  40.0  2.0    1.0   >50K
State: FL, df Length: 16000
   AGEP  COW  SCHL  MAR    OCCP  POBP  RELP  WKHP  SEX  RAC1P  PINCP
0  56.0  3.0  19.0  1.0    10.0  36.0   0

In [9]:
for state_code, df in dfs_loaded.items():
    df_temp=dfs_loaded[state_code].reset_index(drop=True)
    train_data, test_data = train_test_split(df_temp, test_size=0.2, random_state=42)
    train_data.to_csv(f'50_clients_data/raw_data/{state_code}.data', header=False, index=False)
    
    test_data['PINCP'] = test_data['PINCP'].astype(str) + '.'
    print(test_data.head())

    test_data.to_csv(f'50_clients_data/raw_data/{state_code}.test', header=False, index=False)
    
    # df_temp.to_csv(f'50_clients_data/raw_data/{state_code}.csv', header=False, index=False)

      AGEP  COW  SCHL  MAR    OCCP   POBP  RELP  WKHP  SEX  RAC1P   PINCP
8756  53.0  1.0  21.0  1.0   800.0    8.0   0.0  40.0  2.0    1.0   >50K.
4660  23.0  1.0  18.0  5.0  4130.0    6.0  17.0  40.0  1.0    1.0  <=50K.
6095  25.0  1.0  21.0  5.0  3401.0    6.0   2.0  36.0  1.0    6.0  <=50K.
304   47.0  3.0  18.0  1.0  2545.0  303.0   0.0  25.0  2.0    1.0  <=50K.
8241  53.0  1.0  21.0  3.0  1640.0  136.0   0.0  50.0  1.0    1.0   >50K.
      AGEP  COW  SCHL  MAR    OCCP  POBP  RELP  WKHP  SEX  RAC1P   PINCP
8756  42.0  1.0  18.0  5.0  4220.0  12.0   0.0  36.0  1.0    1.0  <=50K.
4660  64.0  1.0  18.0  1.0   710.0  12.0   0.0  20.0  1.0    1.0  <=50K.
6095  33.0  1.0  20.0  5.0  5740.0  36.0   2.0  14.0  2.0    1.0  <=50K.
304   51.0  1.0  21.0  1.0  5000.0  36.0   0.0  30.0  2.0    1.0  <=50K.
8241  48.0  1.0  21.0  1.0  8740.0  12.0   0.0  40.0  2.0    1.0   >50K.
      AGEP  COW  SCHL  MAR    OCCP  POBP  RELP  WKHP  SEX  RAC1P   PINCP
8756  61.0  4.0  22.0  1.0  2205.0  13.0   0.

## Torch2 -- Group and Subgroup

In [8]:
merge_df=merge_dfs["NY"].copy()

In [9]:
col_names=['AGEP', 'COW', 'SCHL', 'MAR', 'OCCP', 'POBP', 'RELP', 'WKHP', 'SEX', 'RAC1P', 'PINCP']

In [10]:
state_code="NY"
temp_df_train=pd.read_csv(f'50_clients_data/raw_data/{state_code}.data', header=None)
temp_df_test=pd.read_csv(f'50_clients_data/raw_data/{state_code}.test', header=None)

temp_df_train.columns = col_names
temp_df_test.columns = col_names
temp_df_train.head()

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,PINCP
0,32.0,1.0,19.0,1.0,410.0,36.0,0.0,45.0,1.0,1.0,>50K
1,44.0,1.0,16.0,1.0,4020.0,36.0,0.0,30.0,2.0,1.0,<=50K
2,37.0,2.0,20.0,1.0,3255.0,365.0,0.0,40.0,2.0,1.0,>50K
3,75.0,6.0,19.0,1.0,1010.0,13.0,0.0,10.0,1.0,1.0,<=50K
4,25.0,1.0,22.0,5.0,440.0,36.0,2.0,40.0,2.0,2.0,<=50K


In [12]:
len(temp_df_train)

12800

In [11]:
temp_df_test.head()

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,PINCP
0,49.0,1.0,16.0,5.0,9620.0,36.0,13.0,35.0,1.0,1.0,<=50K.
1,54.0,1.0,12.0,5.0,3603.0,36.0,13.0,40.0,2.0,1.0,<=50K.
2,53.0,6.0,9.0,1.0,4600.0,329.0,0.0,40.0,2.0,8.0,<=50K.
3,24.0,1.0,19.0,5.0,1970.0,8.0,2.0,20.0,2.0,2.0,<=50K.
4,27.0,1.0,21.0,5.0,630.0,36.0,4.0,45.0,2.0,5.0,<=50K.


In [13]:
merge_df_train =temp_df_train
merge_df_test =temp_df_test
len(merge_df_train),len(merge_df_test)

(12800, 3200)

In [14]:
merge_df_train.head()

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,PINCP
0,32.0,1.0,19.0,1.0,410.0,36.0,0.0,45.0,1.0,1.0,>50K
1,44.0,1.0,16.0,1.0,4020.0,36.0,0.0,30.0,2.0,1.0,<=50K
2,37.0,2.0,20.0,1.0,3255.0,365.0,0.0,40.0,2.0,1.0,>50K
3,75.0,6.0,19.0,1.0,1010.0,13.0,0.0,10.0,1.0,1.0,<=50K
4,25.0,1.0,22.0,5.0,440.0,36.0,2.0,40.0,2.0,2.0,<=50K


In [15]:
#White Men
WM = merge_df_train.loc[(merge_df_train['SEX'] == 1) & (merge_df_train['RAC1P'] == 1)]
#Black Men
BM = merge_df_train.loc[(merge_df_train['SEX'] == 1) & (merge_df_train['RAC1P'] == 2)]
#white Women
WW = merge_df_train.loc[(merge_df_train['SEX'] == 2) & (merge_df_train['RAC1P'] == 1)]
#Black Women
BW = merge_df_train.loc[(merge_df_train['SEX'] == 2) & (merge_df_train['RAC1P'] == 2)]

print("Length: ",len(WM),len(BM),len(WW),len(BW))

WW.to_csv(f'50_clients_data/client_subG_splits/WW.data', header=False, index=False)
WM.to_csv(f'50_clients_data/client_subG_splits/WM.data', header=False, index=False)
BW.to_csv(f'50_clients_data/client_subG_splits/BW.data', header=False, index=False)
BM.to_csv(f'50_clients_data/client_subG_splits/BM.data', header=False, index=False)

Length:  4715 636 4465 800


In [16]:
WM.head()

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,PINCP
0,32.0,1.0,19.0,1.0,410.0,36.0,0.0,45.0,1.0,1.0,>50K
3,75.0,6.0,19.0,1.0,1010.0,13.0,0.0,10.0,1.0,1.0,<=50K
16,51.0,1.0,19.0,1.0,540.0,36.0,0.0,40.0,1.0,1.0,>50K
17,24.0,1.0,16.0,5.0,4700.0,36.0,2.0,40.0,1.0,1.0,<=50K
23,58.0,3.0,23.0,1.0,2100.0,36.0,0.0,40.0,1.0,1.0,>50K


In [17]:
# # # filter the dataframe for each combination

# #White Men
# WM_test = merge_df_test.loc[(merge_df_test['SEX'] == 1) & (merge_df_test['RAC1P'] == 1)]
# #Black Men
# BM_test = merge_df_test.loc[(merge_df_test['SEX'] == 1) & (merge_df_test['RAC1P'] == 2)]
# #white Women
# WW_test = merge_df_test.loc[(merge_df_test['SEX'] == 2) & (merge_df_test['RAC1P'] == 1)]
# #Black Women
# BW_test = merge_df_test.loc[(merge_df_test['SEX'] == 2) & (merge_df_test['RAC1P'] == 2)]

# print("Length: ",len(WM_test),len(BM_test),len(WW_test),len(BW_test))

# WW_test.to_csv(f'50_clients_data/client_subG_splits/WW.test', header=False, index=False)
# WM_test.to_csv(f'50_clients_data/client_subG_splits/WM.test', header=False, index=False)
# BW_test.to_csv(f'50_clients_data/client_subG_splits/BW.test', header=False, index=False)
# BM_test.to_csv(f'50_clients_data/client_subG_splits/BM.test', header=False, index=False)

# BM_test.head()

In [18]:
# WM_test['PINCP'] = WM_test['PINCP'].astype(str) + '.'


In [19]:

# # =====Training=============

# WW_cut= 111
# WM_cut= 111
# BW_cut=111
# BM_cut=111

# # =====Testing=============


# WW.loc[WW_cut:, 'PINCP'] = WW.loc[WW_cut:, 'PINCP'].astype(str) + '.'
# WW[WW_cut:].to_csv(f'50_clients_data/client_subG_splits/WW.test', header=False, index=False)

# WM.loc[WM_cut:, 'PINCP'] = WM.loc[WM_cut:, 'PINCP'].astype(str) + '.'
# WM[WM_cut:].to_csv(f'50_clients_data/client_subG_splits/WM.test', header=False, index=False)

# BW.loc[BW_cut:, 'PINCP'] = BW.loc[BW_cut:, 'PINCP'].astype(str) + '.'
# BW[BW_cut:].to_csv(f'50_clients_data/client_subG_splits/BW.test', header=False, index=False)

# BM.loc[BM_cut:, 'PINCP'] = BM.loc[BM_cut:, 'PINCP'].astype(str) + '.'
# BM[BM_cut:].to_csv(f'50_clients_data/client_subG_splits/BM.test', header=False, index=False)


# WW[WW_cut:].head()


#-------------- Group data with NY-------------
### subgroup data with NY

In [12]:
merge_df=merge_dfs["NY"].copy()
len(merge_df)

103021

In [13]:
men = merge_df.loc[(merge_df['SEX'] == 1)]
women = merge_df.loc[(merge_df['SEX'] == 2)]
white = merge_df.loc[(merge_df['RAC1P'] == 1)]
black = merge_df.loc[(merge_df['RAC1P'] == 2)]

print("Length: ",len(men),len(women),len(white),len(black))

# =====Training=============
men[:40000].to_csv(f'50_clients_data/client_subG_splits/men.data', header=False, index=False)
women[:40000].to_csv(f'50_clients_data/client_subG_splits/women.data', header=False, index=False)

white[:40000].to_csv(f'50_clients_data/client_subG_splits/white.data', header=False, index=False)
black[:10000].to_csv(f'50_clients_data/client_subG_splits/black.data', header=False, index=False)


# =====Testing=============


men.loc[40000:, 'PINCP'] = men.loc[40000:, 'PINCP'].astype(str) + '.'
men[40000:].to_csv(f'50_clients_data/client_subG_splits/men.test', header=False, index=False)

women.loc[40000:, 'PINCP'] = women.loc[40000:, 'PINCP'].astype(str) + '.'
women[40000:].to_csv(f'50_clients_data/client_subG_splits/women.test', header=False, index=False)

white.loc[40000:, 'PINCP'] = white.loc[40000:, 'PINCP'].astype(str) + '.'
white[40000:].to_csv(f'50_clients_data/client_subG_splits/white.test', header=False, index=False)

black.loc[10000:, 'PINCP'] = black.loc[10000:, 'PINCP'].astype(str) + '.'
black[10000:].to_csv(f'50_clients_data/client_subG_splits/black.test', header=False, index=False)

(52178, 50843, 73665, 11647)

# Take random 100 data points from Clients

In [13]:
import os

In [14]:
# Training Ground Truth

In [4]:
folder_path = '50_clients_data/raw_data/'
for file_name in os.listdir(folder_path):
    if file_name.endswith('.data'):
        df = pd.read_csv(os.path.join(folder_path, file_name), header=None)
        temp_df=df.sample(n=100)
        base_name = os.path.splitext(file_name)[0]
        temp_df.to_csv(f'50_clients_data/client_raw_data_100_sample/{base_name}_100.data', header=False, index=False)


In [16]:
# Testing

In [5]:
folder_path = '50_clients_data/raw_data/'
for file_name in os.listdir(folder_path):
    if file_name.endswith('.test'):
        df = pd.read_csv(os.path.join(folder_path, file_name), header=None)
        temp_df=df.sample(n=100)
        base_name = os.path.splitext(file_name)[0]
        temp_df.to_csv(f'50_clients_data/client_raw_data_100_test/{base_name}_100.test', header=False, index=False)

In [19]:
# for state_code, df in dfs_loaded.items():
#     df_temp=dfs_loaded[state_code].reset_index(drop=True)
#     train_data, test_data = train_test_split(df_temp, test_size=0.2, random_state=42)
    
#     train_data.to_csv(f'50_clients_data/client_raw_data_100_sample/{state_code}_100.data', header=False, index=False)

# sample_100_AK.to_csv(f'50_clients_data/raw_data/AK_100.data', header=False, index=False)

In [21]:
# for state_code, (features, label) in dfs.items():
#     # take 30%
#     num_rows_to_keep = int(len(features) * 0.3) 
#     random_indices = np.random.choice(len(features), num_rows_to_keep, replace=False)
#     reduced_features = features.iloc[random_indices]
#     reduced_label = label.iloc[random_indices]
#     dfs[state_code] = (reduced_features, reduced_label)

# for state_code, (reduced_features, reduced_label) in dfs.items():
#     print(f"State: {state_code}, Reduced Features Length: {len(reduced_features)}, Reduced Label Length: {len(reduced_label)}")

In [16]:
import pickle

# Save the dictionary to a file
with open('dfs.pickle', 'wb') as f:
    pickle.dump(merge_dfs, f)

In [17]:
with open('dfs.pickle', 'rb') as f:
    dfs_loaded = pickle.load(f)

In [18]:
for state_code, df in dfs_loaded.items():
    print(f"State: {state_code}, df Length: {len(df)}")

State: AL, df Length: 22268


In [20]:
dfs_loaded["AL"].reset_index(drop=True)

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,PINCP
0,18.0,1.0,18.0,5.0,4720.0,13.0,17.0,21.0,2.0,2.0,<=50K
1,53.0,5.0,17.0,5.0,3605.0,18.0,16.0,40.0,1.0,1.0,<=50K
2,41.0,1.0,16.0,5.0,7330.0,1.0,17.0,40.0,1.0,1.0,<=50K
3,18.0,6.0,18.0,5.0,2722.0,1.0,17.0,2.0,2.0,1.0,<=50K
4,21.0,5.0,19.0,5.0,3870.0,12.0,17.0,50.0,1.0,1.0,<=50K
...,...,...,...,...,...,...,...,...,...,...,...
22263,20.0,6.0,19.0,5.0,4251.0,1.0,4.0,25.0,1.0,1.0,<=50K
22264,63.0,1.0,16.0,1.0,440.0,1.0,0.0,48.0,1.0,1.0,>50K
22265,65.0,2.0,21.0,5.0,420.0,1.0,2.0,40.0,2.0,1.0,>50K
22266,37.0,1.0,16.0,4.0,340.0,6.0,0.0,50.0,2.0,1.0,<=50K
